In [ ]:
!apt update
#!apt upgrade -y
!uname -m && cat /etc/*release
!gcc --version
!uname -r


In [ ]:
#Installing Darknet
!git clone https://github.com/AlexeyAB/darknet


In [ ]:
#Activating CUDA and GPU
%cd /content/darknet/
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!apt update
!apt-get install libopencv-dev

In [ ]:
!/usr/local/cuda/bin/nvcc --version

In [ ]:
!make

In [ ]:
#Checking whether the GPU is installed.
!nvidia-smi

In [ ]:
#Upload yolov4.weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
#Activating Helper Functions(These three functions are helper functions that will allow you to show the image in your Colab Notebook after running your detections, as well as upload and download images to and from your Cloud VM.)
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!unzip /content/gdrive/MyDrive/visdrone_data/VisDrone2019-DET-train.zip -d /content/darknet/data




In [ ]:
%cd /content/darknet/data
!mv VisDrone2019-DET-train Train

In [ ]:
!cp /content/gdrive/MyDrive/visdrone_data/yolov4.conv.137 /content/darknet/
!cp /content/gdrive/MyDrive/visdrone_data/yolov4_VisDrone_Train.cfg /content/darknet/cfg/
!cp /content/gdrive/MyDrive/visdrone_data/yolov4_VisDrone_Test.cfg /content/darknet/cfg/




In [ ]:
import os
from pathlib import Path
import PIL
from PIL import Image

# Get the list of all files and directories
# in the root directory
path1 = "/content/darknet/data/Train/images/"
path2 = "/content/darknet/data/Train/annotations/"

dir_list = os.listdir(path1)
d2 = os.listdir(path2)

for image in dir_list:
    img_path = str(path1 + str(image))
    # print(img_path)
    img = PIL.Image.open(img_path)
    wx, hx = img.size
    temp = "/content/darknet/data/Train"+"/annotations/" + str(image)
    pstring = temp.replace('.jpg', '.txt')
    file = Path(pstring)
    print(file)
    input_lines = file.read_text().split('\n')
    input_lines.pop();

    ouput_lines = []

    for input_line in input_lines:
        input_line = input_line.split(',')
        if int(input_line[4]):
            x = int(input_line[0])
            y = int(input_line[1])
            w = int(input_line[2])
            h = int(input_line[3])
            xc = x + w / 2
            yc = y + h / 2
            xc /= wx
            yc /= hx
            w /= wx
            h /= hx
            ouput_lines.append(' '.join(
                [input_line[5]] + [str(format(xc, ".4f"))] + [str(format(yc, ".4f"))] + [str(format(w, ".4f"))] + [
                    str(format(h, ".4f"))]))
    temp = "/content/darknet/data/Train"+"/images/" + str(image)
    pstring = temp.replace('.jpg', '.txt')
    file = Path(pstring)
    file.write_text('\n'.join(ouput_lines))


In [ ]:
# Creating files train.txt and test.txt
# for training in Darknet framework
#
# Algorithm:
# Setting up full path --> List of paths -->
# --> Extracting 15% of paths to save into test.txt file -->
# --> Writing paths into train and test txt files
#
# Result:
# Files train.txt and test.txt with full paths to images


import os


"""
Start of:
Setting up full path to directory with VisDrone images
"""

full_path_to_images = '/content/darknet/data/Train/images'

"""
End of:
Setting up full path to directory with VisDrone images
"""


"""
Start of:
Getting list of full paths to downloaded images
"""

# Changing the current directory
# to one with images
os.chdir(full_path_to_images)

# Defining list to write paths in
p = []

# Using os.walk for going through all directories
# and files in them from the current directory
# Fullstop in os.walk('.') means the current directory
for current_dir, dirs, files in os.walk('.'):
    # Going through all files
    for f in files:
        # Checking if filename ends with '.jpg'
        if f.endswith('.jpg'):
            # Preparing path to save into train.txt file
            
            path_to_save_into_txt_files = full_path_to_images + '/' + f

            # Appending the line into the list
            # We use here '\n' to move to the next line
            # when writing lines into txt files
            p.append(path_to_save_into_txt_files + '\n')


# Slicing first 15% of elements from the list
# to write into the test.txt file
p_test = p[:int(len(p) * 0.15)]

# Deleting from initial list first 15% of elements
p = p[int(len(p) * 0.15):]

"""
End of:
Getting list of full paths to downloaded images
"""


"""
Start of:
Creating train.txt and test.txt files
"""

# Creating file train.txt and writing 85% of lines in it
with open('/content/darknet/data/train.txt', 'w') as train_txt:
    # Going through all elements of the list
    for e in p:
        # Writing current path at the end of the file
        train_txt.write(e)

# Creating file test.txt and writing 15% of lines in it
with open('/content/darknet/data/test.txt', 'w') as test_txt:
    # Going through all elements of the list
    for e in p_test:
        # Writing current path at the end of the file
        test_txt.write(e)

"""
End of:
Creating train.txt and test.txt files
"""


In [ ]:
!cp /content/gdrive/MyDrive/visdrone_data/Classes.txt /content/darknet/data/Train/images

In [ ]:
# Creating files VisDrone.data and classes.names
# for training in Darknet framework
#Create a classes.txt  and classes.names file beforehand
# Algorithm:
# Setting up full path --> Reading file classes.txt -->
# --> Creating file classes.names -->
# --> Creating file ts_data.data
#
# Result:
# Files classes.names and ts_data.data needed to train
# in Darknet framework


"""
Start of:
Setting up full path to directory with VisDrone images
"""

full_path_to_images = '/content/darknet/data/Train/images'

"""
End of:
Setting up full path to directory with VisDrone images
"""


"""
Start of:
Creating file classes.names
"""

# Defining counter for classes
c = 0


with open('/content/darknet/data' + '/' + 'classes.names', 'w') as names, \
     open(full_path_to_images + '/' + 'Classes.txt', 'r') as txt:

    # Going through all lines in txt file and writing them into names file
    for line in txt:
        names.write(line)  # Copying all info from file txt to names

        # Increasing counter
        c += 1

"""
End of:
Creating file classes.names
"""


"""
Start of:
Creating file VisDrone.data
"""

with open('/content/darknet/cfg' + '/' + 'VisDrone.data', 'w') as data:
    # Writing needed 5 lines
    # Number of classes
    # By using '\n' we move to the next line
    data.write('classes = ' + str(c) + '\n')

    # Location of the train.txt file
    data.write('train = data/train.txt' + '\n')

    # Location of the test.txt file
    data.write('valid = data/test.txt' + '\n')

    # Location of the classes.names file
    data.write('names = data/classes.names' + '\n')

    # Location where to save weights
    data.write('backup = backup')

"""
End of:
Creating file VisDrone.data
"""


In [ ]:
#Training the data with the custom trained weights

!./darknet detector train cfg/VisDrone.data cfg/yolov4_VisDrone_Train.cfg yolov4.conv.137 -dont_show



In [ ]:
%cd /content/darknet/
!./darknet detector test cfg/VisDrone.data cfg/yolov4_VisDrone_Test.cfg /content/gdrive/MyDrive/visdrone_data/yolov4_VisDrone_Train_final.weights /content/gdrive/MyDrive/visdrone_data/0000001_02999_d_0000005.jpg